# Load Libraries and Data

In [1]:
DEBUG = True
TO_KAGGLE = False
file_name = "006"
MEMO = "10folds,embedding003,004を使用"
model = "svr"
score_path = "/home/jupyter/output/scores/scores003.csv"

In [2]:
import os
import datetime
import pickle
import glob

import numpy as np 
import pandas as pd 
import os, gc, re, warnings
warnings.filterwarnings("ignore")

from transformers import AutoModel,AutoTokenizer
import torch
import torch.nn.functional as F
from tqdm import tqdm

model_name = "svr"
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
now = datetime.datetime.now(JST)
date = now.strftime('%Y%m%d')
date2 = now.strftime('%Y%m%d%H%M')


path ="/home/jupyter/feedback-prize-english-language-learning/"
if DEBUG:
    OUTPUT_DIR = f'/home/jupyter/output/ex/DEBUG/{model_name}/{file_name}/{date2}/'
else:
    OUTPUT_DIR = f'/home/jupyter/output/ex/{model_name}/{file_name}/{date2}/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [3]:
target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions',]

dftr = pd.read_csv("/home/jupyter/feedback-prize-english-language-learning/train.csv")
dftr["src"]="train"
dfte = pd.read_csv("/home/jupyter/feedback-prize-english-language-learning/test.csv")
dfte["src"]="test"
print('Train shape:',dftr.shape,'Test shape:',dfte.shape,'Test columns:',dfte.columns)
df = pd.concat([dftr,dfte],ignore_index=True)

fold = pd.read_csv("/home/jupyter/output/fold/10folds.csv")
fold.rename(columns={"fold":"FOLD"},inplace=True)

dftr = pd.merge(dftr,fold,how="left",on="text_id")
if DEBUG:
    dftr = dftr.head(20)

Train shape: (3911, 9) Test shape: (3, 3) Test columns: Index(['text_id', 'full_text', 'src'], dtype='object')


# Combine Feature Embeddings

In [4]:
dir_path = '/home/jupyter/output/ex/embedding/003/'
files_path = []
for current_dir, sub_dirs, files_list in os.walk(dir_path): 
    for file in files_list: 
        files_path.append(os.path.join(current_dir,file))

emb_path = [s for s in files_path if ("all_embedding.npy" not in s) and ("npy" in s) and ("mean.npy" not in s)]
cfg_list = [s.split("/")[-1].split("_")[0] for s in emb_path]
cfg_list = list(set(cfg_list))

In [5]:
cfg_list

['CFG1', 'CFG2', 'CFG3', 'CFG5', 'CFG4', 'CFG9', 'CFG8', 'CFG6']

In [19]:
dftr.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,src,FOLD,pred_cohesion,pred_syntax,pred_vocabulary,pred_phraseology,pred_grammar,pred_conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,train,9,NaN,NaN,NaN,NaN,NaN,NaN
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,train,2,2.871366,2.935812,3.052612,2.734635,2.775023,2.529649
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,train,1,3.197907,3.242587,3.245298,3.189578,3.204165,2.920316
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,train,1,3.419058,3.399498,3.388095,3.488686,3.440051,3.197580
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,train,0,2.916285,2.847148,3.125768,2.692136,2.839136,2.584031


In [9]:
dir_path = '/home/jupyter/output/ex/embedding/004/'
files_path = []
for current_dir, sub_dirs, files_list in os.walk(dir_path): 
    for file in files_list: 
        files_path.append(os.path.join(current_dir,file))
oof_path = [s for s in files_path if "oof_df.pkl" in s]


In [30]:
fold = 0
all_oof_feats = pd.read_pickle(oof_path[1])
all_oof_feats = pd.merge(dftr["text_id"],all_oof_feats,how="left",on="text_id")


In [28]:
all_oof_feats.loc[all_oof_feats.fold==0]

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,fold,pred_cohesion,pred_syntax,pred_vocabulary,pred_phraseology,pred_grammar,pred_conventions
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,0,2.427323,2.465084,2.687973,2.440547,2.555911,2.336514
13,00ED2563D0B1,"Philosopher, physician, and humanitarian Alber...",3.5,3.0,3.5,3.5,3.5,4.0,0,3.800885,3.851415,3.941611,3.819339,3.906917,3.766028


In [16]:
fold = 0

all_oof_feats = pd.read_pickle(oof_path[0])
for path in oof_path[1:]:
    oof = pd.read_pickle(path)
    all_oof_feats = np.concatenate([all_oof_feats,oof],axis=1)



ValueError: Cannot load file containing pickled data when allow_pickle=False

# Train RAPIDS cuML SVR
Documentation for RAPIDS SVM is [here][1]

[1]: https://docs.rapids.ai/api/cuml/stable/api.html#support-vector-machines

In [6]:
from cuml.svm import SVR
import cuml
print('RAPIDS version',cuml.__version__)

RAPIDS version 21.10.02


In [7]:
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores

def get_result2(oof_df):
    labels = oof_df[target_cols].values
    preds = oof_df[[f"pred_{c}" for c in target_cols]].values
    score, scores = get_score(labels, preds)
    #LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
    return score,scores

In [8]:
from sklearn.metrics import mean_squared_error
import joblib

scores = []
def comp_score(y_true,y_pred):
    rmse_scores = []
    for i in range(len(target_cols)):
        rmse_scores.append(np.sqrt(mean_squared_error(y_true[:,i],y_pred[:,i])))
    return np.mean(rmse_scores)

#for fold in tqdm(range(FOLDS),total=FOLDS):
for fold in range(dftr.FOLD.nunique()):
    print('#'*25)
    print('### Fold',fold+1)
    print('#'*25)
    
    dftr_ = dftr[dftr["FOLD"]!=fold]
    dfev_ = dftr[dftr["FOLD"]==fold]
    
    _ = [s for s in emb_path if f"fold{fold}" in s]
    all_train_text_feats = np.load(_[0])
    for path in _[1:]:
        emb = np.load(path)
        all_train_text_feats = np.concatenate([all_train_text_feats,emb],axis=1)

    if DEBUG:
        print()
        all_train_text_feats = all_train_text_feats[:20]

    tr_text_feats = all_train_text_feats[list(dftr_.index),:]
    ev_text_feats = all_train_text_feats[list(dfev_.index),:]
    
    ev_preds = np.zeros((len(ev_text_feats),6))
    for i,t in enumerate(target_cols):
        print(t,', ',end='')
        modelname = OUTPUT_DIR+f'svr_{t}_fold{fold}.sav'
        clf = SVR(C=1)
        clf.fit(tr_text_feats, dftr_[t].values)
        joblib.dump(clf, modelname)
        loaded_model = joblib.load(modelname)
        ev_preds[:,i] = loaded_model .predict(ev_text_feats)
        
    cols = [f"pred_{t}" for t in target_cols]
    dftr.loc[list(dfev_.index),cols] = ev_preds
    print()
    score = comp_score(dfev_[target_cols].values,ev_preds)
    scores.append(score)
    print("Fold : {} RSME score: {}".format(fold,score))

dftr.to_pickle(OUTPUT_DIR+f'oof_df.pkl')
score,scores = get_result2(dftr)
print('#'*25)
print(score)

#########################
### Fold 1
#########################

cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 0 RSME score: 0.3149222462188594
#########################
### Fold 2
#########################

cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 1 RSME score: 0.7744570741558268
#########################
### Fold 3
#########################

cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 2 RSME score: 0.4339269762270967
#########################
### Fold 4
#########################

cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 3 RSME score: 0.47482786198442595
#########################
### Fold 5
#########################

cohesion , 

RuntimeError: UnownedMemory requires explicit device ID for a null pointer.

In [ ]:
if TO_KAGGLE:
    UPLOAD_DIR = OUTPUT_DIR
    EX_NO = f"{model}-{file_name}" # 実験番号などを入れる、folderのpathにする
    USERID = 'your_id'


    def dataset_upload():
        import json
        from kaggle.api.kaggle_api_extended import KaggleApi

        id = f'{USERID}/{EX_NO}'

        dataset_metadata = {}
        dataset_metadata['id'] = id
        dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
        dataset_metadata['title'] = f'{EX_NO}'

        with open(UPLOAD_DIR +'dataset-metadata.json', 'w') as f:
            json.dump(dataset_metadata, f, indent=4)

        api = KaggleApi()
        api.authenticate()

        # データセットがない場合
        if f'{USERID}/{EX_NO}' not in [str(d) for d in api.dataset_list(user=USERID, search=f'"{EX_NO}"')]:
            api.dataset_create_new(folder=UPLOAD_DIR,
                                   convert_to_csv=False,
                                   dir_mode='skip')
            
            #apiコマンドを書き込む
            f = open(f'{model}_api_command.txt', 'a')
            api_command = f"!kaggle datasets download -d hiroki8383/{EX_NO}\n"
            f.write(api_command)
            f.close()
            
            #フォルダーを削除
            if f'{USERID}/{EX_NO}' not in [str(d) for d in api.dataset_list(user=USERID, search=f'"{EX_NO}"')]:
                remove_files = glob.glob(OUTPUT_DIR+"*")
                remove_files.remove(OUTPUT_DIR+"oof_df.pkl")
                for file in remove_files:
                    os.remove(file)
                print("folder upload")
            else:
                print("folder not upload")
            
            
        # データセットがある場合→更新されない場合がある（後で原因追及)
        else:
            print("this folder exsits")
            # api.dataset_create_version(folder=UPLOAD_DIR,
            #                            version_notes='update',
            #                            convert_to_csv=False,
            #                            delete_old_versions=False,
            #                            dir_mode='zip')

        

        
    dataset_upload()

In [ ]:
print(OUTPUT_DIR)

In [ ]:
MEMO = "10folds,embedding003を使用"

In [ ]:
if not DEBUG:
    def to_write_score(MEMO,score_path):
        df = pd.read_csv(score_path)
        def get_result2(oof_df):
                labels = oof_df[target_cols].values
                preds = oof_df[[f"pred_{c}" for c in target_cols]].values
                score, scores = get_score(labels, preds)
                #LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
                return score,scores

        score,scores = get_result2(dftr)
        name = "-".join(OUTPUT_DIR.split("/")[-4:-2])
        base = {"name":name,"score":score,"memo":MEMO} 
        base.update(dict(zip(target_cols,scores)))
        df = df.append(base,ignore_index=True).sort_values(by="name").reset_index(drop=True)
        df.to_csv(score_path,index=False)
    to_write_score(MEMO,score_path)

In [ ]:
df = pd.read_csv(score_path)
df

In [ ]:
df.sort_values(ascending=True,by="score")

In [ ]:
df[df.name.isin([n for n in df.name if "roberta" in n])].sort_values(ascending=True,by="score")